## Intégrer Refashion depuis l'api pointsapport :

- Récupérer les données-eo-refashion depuis l'api pointsapport.
- Créer et mapper les données vers les tables Acteurs, Proposition de Services et Sous-catégories.
- Enregistrer chaque table dans un fichier CSV.

#### préprod

In [150]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Accessing environment variables
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  # Default PostgreSQL port is 5432, but we're using a custom one here
db_name = os.getenv('DB_NAME')

# Create the connection URL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'

# Create the engine
engine = create_engine(connection_string)   


#### prod

In [152]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Accessing environment variables
user = os.getenv('DB_USER_PROD')
password = os.getenv('DB_PASSWORD_PROD')
host = os.getenv('DB_HOST_PROD')
port = os.getenv('DB_PORT_PROD')  # Default PostgreSQL port is 5432, but we're using a custom one here
db_name = os.getenv('DB_NAME_PROD')

# Create the connection URL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'

# Create the engine
engine_prod = create_engine(connection_string)


## Get data from point apport 

In [ ]:
import requests
import pandas as pd


def fetch_all_data(url):
    all_data = []
    while url:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data['results'])
            # Check if there's a next page link
            url = data.get('next', None)
            print(url)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break
    return all_data

api_url = "https://data.pointsapport.ademe.fr/data-fair/api/v1/datasets/donnees-eo-refashion/lines?size=10000"

data = fetch_all_data(api_url)

df = pd.DataFrame(data)



In [ ]:
df_acteurtype = pd.read_sql_table('qfdmo_acteurtype', engine)
df_sources = pd.read_sql_table('qfdmo_source', engine)
df_da = pd.read_sql_table('qfdmo_displayedacteur', engine)
df_ps = pd.read_sql_table('qfdmo_propositionservice', engine)
df_ps['id'].max()
df_pssc = pd.read_sql_table('qfdmo_propositionservice_sous_categories', engine)
df_action = pd.read_sql_table('qfdmo_action', engine)
df_ac = pd.read_sql_table('qfdmo_acteur', engine)


In [ ]:
df_action = pd.read_sql_table('qfdmo_action', engine)
engine

In [ ]:
df_action

### Mappers

In [ ]:
column_mapping = {
    'id_point_apport_ou_reparation': 'identifiant_externe',
    'adresse_complement': 'adresse_complement',
    'type_de_point_de_collecte': 'acteur_type_id',
    'telephone': 'telephone',
    'siret': 'siret',
    'uniquement_sur_rdv': '',
    'exclusivite_de_reprisereparation': '',
    'filiere': '',
    'public_accueilli': '',
    'produitsdechets_acceptes': '',
    'labels_etou_bonus': '',
    'reprise': '',
    'point_de_reparation': '',
    'ecoorganisme': 'source_id',
    'adresse_format_ban': 'adresse',
    'nom_de_lorganisme': 'nom',
    'enseigne_commerciale':'nom_commercial',
    '_updatedAt':'cree_le',
    'site_web': 'url',
    'email': 'email',
    'perimetre_dintervention': '',
    'longitudewgs84': 'location',  
    'latitudewgs84': 'location',  
    'horaires_douverture': 'horaires',
    'consignes_dacces': 'description',
}


# Print the dictionary for visual confirmation
print(column_mapping)

### Transformations

#### Create Actors

In [ ]:
from shapely.geometry import Point
from shapely import wkb
import re
import hashlib


selected_columns = ['nom', 'adresse', 'type_de_point_de_collecte', 'id_point_apport_ou_reparation','identifiant_externe']

def generate_unique_id(row):
    unique_str = '_'.join([str(row[col]) for col in selected_columns])
    return hashlib.sha256(unique_str.encode()).hexdigest()
def transform_acteur_type_id(value):
    mapping_dict = {
        "Solution en ligne (site web, app. mobile)": "en ligne (web, mobile)",
        "Artisan, commerce indépendant": "artisan, commerce indépendant",
        "Magasin / Franchise, Enseigne commerciale / Distributeur / Point de vente": "commerce",
        "Point d'Apport Volontaire Publique": "point d'apport volontaire public",
        "Association, entreprise de l’économie sociale et solidaire (ESS)": "Association, entreprise de l'ESS",
        "Déchèterie": "déchèterie",
    }
    nom_affiche = mapping_dict.get(value)
    id_value = df_acteurtype.loc[df_acteurtype['nom_affiche'] == nom_affiche, 'id'].values[0] if any(df_acteurtype['nom_affiche'] == nom_affiche) else None
    return id_value



def transform_location(longitude, latitude):
    point = Point(longitude, latitude)
    
    transformed_location_binary = wkb.dumps(point)
    transformed_location_hex = transformed_location_binary.hex()

    return transformed_location_hex

def transform_ecoorganisme(value):
    
    id_value = df_sources.loc[df_sources['nom'].str.lower() == value.lower(), 'id'].values[0] if any(df_sources['nom'].str.lower() == value.lower()) else None
    return id_value

def extract_details(row):
    pattern = re.compile(r'\b(\d{5})\s+(.*)')
    
    address = None
    postal_code = None
    city = None
    if pd.isnull(row['adresse_format_ban']):
        return pd.Series([None, None, None])

    # Ensure adress_ban is treated as a string
    adress_ban = str(row['adresse_format_ban'])
    
    # Search for the pattern
    match = pattern.search(adress_ban)
    if match:
        postal_code = match.group(1)
        city = match.group(2)
        address = adress_ban[:match.start()].strip()
    
    return pd.Series([address, postal_code, city])

# Apply the function and assign the result to new columns
for old_col, new_col in column_mapping.items():
    if new_col: 
        if old_col == 'type_de_point_de_collecte':
            df[new_col] = df[old_col].apply(transform_acteur_type_id)
        elif old_col in ('longitudewgs84', 'latitudewgs84'):
            df['location'] = df.apply(lambda row: transform_location(row['longitudewgs84'], row['latitudewgs84']), axis=1)
        elif old_col == 'ecoorganisme':
            df[new_col] = df[old_col].apply(transform_ecoorganisme)
        elif old_col == 'adresse_format_ban':
            df[['adresse', 'code_postal', 'ville']] = df.apply(extract_details, axis=1)
        else:
            df[new_col] = df[old_col]
df['label_reparacteur']=False
df['statut']='ACTIF'
df['identifiant_unique'] = df.apply(generate_unique_id, axis=1)
            


In [ ]:
df.loc[df['service_a_domicile']=='service à domicile uniquement','statut'] = 'SUPPRIME'

In [ ]:
df['modifie_le'] = df['cree_le']
df['siret'] = df['siret'].astype(str).apply(lambda x : x[:14])
df['telephone'] = df['telephone'].dropna().apply(lambda x: x.replace(' ', ''))
df['telephone'] = df['telephone'].dropna().apply(lambda x: '0' + x[2:] if x.startswith('33') else x)

In [ ]:
df.drop_duplicates('identifiant_unique', keep='first', inplace=True)

#### Create Proposition de services

In [ ]:
rows_list = []

for index, row in df.iterrows():
    acteur_id = row['identifiant_unique']
    action_id = None
    sous_categories = row['produitsdechets_acceptes']
    if row['point_dapport_de_service_reparation']:
        acteur_service_id = 17
        action_id = 1
    elif row['point_dapport_pour_reemploi']:
        acteur_service_id = 4
        action_id = 4
    elif row['point_de_reparation']:
        acteur_service_id = 15
        action_id = 1
    elif row['point_de_collecte_ou_de_reprise_des_dechets']:
        acteur_service_id = 4
        action_id = 43
    else:
        continue  # Skip rows that don't match any criteria
    
    rows_list.append({"acteur_service_id": acteur_service_id, "action_id": action_id, "acteur_id": acteur_id, "sous_categories":sous_categories})

df_pds = pd.DataFrame(rows_list)
df_pds.index = range(df_ps['id'].max()+1, df_ps['id'].max()+1 + len(df_pds))

df_pds['id'] = df_pds.index


#### Create sous categories

In [ ]:
rows_list=[]
sous_categories = { 
    "Vêtement" : 107,
    "Linge" : 104,
    "Chaussure":109
}
for index, row in df_pds.iterrows():
    products = str(row["sous_categories"]).split("|")
    for product in products:
        if product.strip() in sous_categories:
            rows_list.append({
                'propositionservice_id': row['id'], 
                'souscategorieobjet_id': sous_categories[product.strip()]
            })

df_sous_categories = pd.DataFrame(rows_list, columns=['propositionservice_id', 'souscategorieobjet_id'])

df_sous_categories

#### Add to DB

In [ ]:

df[[
    'identifiant_unique',
    'acteur_type_id',
'adresse',
    'code_postal', 'ville',
 'adresse_complement',
 'commentaires',
    'description',
 'email',
 'horaires',
 'identifiant_externe',
 'label_reparacteur',
 'nom_commercial',
 'nom',
    'location',
'cree_le',
'modifie_le',
'multi_base',
'manuel',
'statut',
 'siret',
 'source_id',
 'telephone',
 'url'
]].to_sql("qfdmo_acteur",engine, if_exists='append',index=False,method='multi',chunksize=1000)

In [ ]:
df_pds[['acteur_service_id','action_id','acteur_id','id']].to_csv('refashion_propositionservice.csv')
df_pds[['id','acteur_service_id','action_id','acteur_id']].to_sql("qfdmo_propositionservice",engine, if_exists='append',index=False,method='multi',chunksize=1000)

In [ ]:
df_sous_categories[['propositionservice_id','souscategorieobjet_id']].to_csv('refashion_sous_categories.csv')

In [ ]:
df_sous_categories.to_sql("qfdmo_propositionservice_sous_categories",engine, if_exists='append',index=False,method='multi',chunksize=1000)

In [ ]:
delete_query= """
DELETE FROM qfdmo_propositionservice_sous_categories
    USING qfdmo_propositionservice_sous_categories_refashion
    WHERE qfdmo_propositionservice_sous_categories.propositionservice_id = qfdmo_propositionservice_sous_categories_refashion.propositionservice_id
    AND qfdmo_propositionservice_sous_categories.souscategorieobjet_id = qfdmo_propositionservice_sous_categories_refashion.souscategorieobjet_id;"""

In [ ]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
user = 
password = 
host = 
port = '33517'  # default PostgreSQL port is 5432
db_name = 'quefairedem_2657'
# Access variables in .env
conn = psycopg2.connect(
    dbname=db_name, 
    user=user, 
    password=password, 
    host=host,
    port=port
)
conn.autocommit = True
cursor = conn.cursor()

query = sql.SQL(sql_query)
cursor.execute(query)

cursor.close()
conn.close()

In [ ]:
df.loc[df['service_a_domicile']=='service à domicile uniquement',['statut','identifiant_unique']].to_sql("qfdmo_acteur_fix_sd",engined)

## Revision Christian --> revisionacteur

In [ ]:
df_chris_rev = pd.read_csv("refashion_acteurs_chris_28032024.csv")

In [136]:
df_chris_rev[['identifiant_unique',
    'acteur_type_id',
'adresse',
    'code_postal', 'ville',
 'adresse_complement',
 'commentaires',
 'email',
 'horaires',
              'description',
 'identifiant_externe',
 'label_reparacteur',
 'nom_commercial',
 'nom',
    'location',
'cree_le',
'modifie_le',
'multi_base',
'manuel',
'statut',
 'siret',
 'source_id',
 'telephone',
 'url']].to_sql("qfdmo_revision_acteur_enrich_christian",engine, if_exists='replace')

930

In [140]:
import psycopg2
from psycopg2 import sql

# Connect to the database again
conn = psycopg2.connect(
    dbname=db_name, 
    user=user, 
    password=password, 
    host=host,
    port=port
)
conn.autocommit = True
cursor = conn.cursor()

# Perform the update
cursor.execute("""
 UPDATE qfdmo_revisionacteur
SET 
    acteur_type_id = NULL,
    adresse = NULL,
    code_postal = NULL, 
    ville = NULL,
    email = NULL,
    horaires = NULL,
    identifiant_externe = NULL,
    label_reparacteur = qfdmo_revision_acteur_enrich_christian.label_reparacteur,
    nom_commercial = NULL,
    nom = NULL,
    location = NULL,
    cree_le = NOW(),
    modifie_le = NOW(),
    statut = qfdmo_revision_acteur_enrich_christian.statut,
    siret = NULL,
    source_id = NULL,
    telephone = NULL,
    description = qfdmo_revision_acteur_enrich_christian.description,
    adresse_complement = qfdmo_revision_acteur_enrich_christian.adresse_complement,
    url = qfdmo_revision_acteur_enrich_christian.url
FROM qfdmo_revision_acteur_enrich_christian
WHERE qfdmo_revisionacteur.identifiant_unique = qfdmo_revision_acteur_enrich_christian.identifiant_unique;

""")

# Cleanup
cursor.close()
conn.close()